<a href="https://colab.research.google.com/github/AlessandraMendes02/Projeto-IA/blob/main/Projeto_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#INSTALAÇÕES
!pip install flask transformers pillow
!pip install flask-ngrok
!pip install pyngrok
!pip install Flask-CORS
!apt-get install -y ngrok

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package ngrok


In [2]:
#IMPORTANDO
from pyngrok import ngrok
from flask import Flask, request, jsonify
from flask_cors import CORS
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import base64
import io

In [3]:
#CONVERTENDO BASE64 EM IMAGEM, TREINANDO O MODELO E RETORNANDO URL DE CONEXÃO
ngrok.set_auth_token("2oysyDs6AlT4zWFCIkoE03un4oA_7yde9saoty62DksqDo5k3")

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})

genereted_url = ngrok.connect(5000)
print(f" * Ngrok URL: {genereted_url}")

@app.route('/process-image', methods=['POST'])
def process_image():
    try:
        data = request.json
        image_base64 = data.get("image")

        if not image_base64:
            return jsonify({"error": "image not found"}), 400

        image_data = base64.b64decode(image_base64.split(",")[1])
        image = Image.open(io.BytesIO(image_data)).convert('RGB')

        inputs = processor(image, return_tensors="pt")
        out = model.generate(**inputs)
        description = processor.decode(out[0], skip_special_tokens=True)

        print(f"Descrição gerada: {description}")

        return jsonify({"description": description})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

app.run(port=5000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


 * Ngrok URL: NgrokTunnel: "https://1d12-35-236-251-95.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 00:18:22] "OPTIONS /process-image HTTP/1.1" 200 -
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 00:18:37] "POST /process-image HTTP/1.1" 200 -


Descrição gerada: woman sitting at a table eating a sandwich and drinking coffee


INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 00:19:35] "OPTIONS /process-image HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/Nov/2024 00:19:51] "POST /process-image HTTP/1.1" 200 -


Descrição gerada: a diagram of a bar chart with a number of different levels
